In [1]:
from datetime import datetime
import os
import torch
from torch import nn

In [3]:
import import_ipynb
from utils import strfdelta

importing Jupyter notebook from utils.ipynb


In [4]:
class EarlyStopping:
    def __init__(self, patience=10, delta=0.0001, project_name=None, checkpoint_file_path=None, run_time_str=None):
        self.patience = patience
        self.counter = 0
        self.delta = delta
        
        self.val_loss_min = None
        self.file_path = os.path.join(
            checkpoint_file_path, f"{project_name}_checkpoint_{run_time_str}.pt"
        )
        self.latest_file_path = os.path.join(
            checkpoint_file_path, f"{project_name}_checkpoint_latest.pt"
        )
    
    def check_and_save(self, new_validation_loss, model):
        early_stop = False
    
        if self.val_loss_min is None:
            self.val_loss_min = new_validation_loss
            message = f'Early stopping is stated!'
        elif new_validation_loss < self.val_loss_min - self.delta:
            message = f'V_loss decreased ({self.val_loss_min:6.3f} --> {new_validation_loss:6.3f}). Saving model...'
            self.save_checkpoint(new_validation_loss, model)
            self.val_loss_min = new_validation_loss
            self.counter = 0
        else:
            self.counter += 1
            message = f'Early stopping counter: {self.counter} out of {self.patience}'
            if self.counter >= self.patience:
                early_stop = True
                message += " *** TRAIN EARLY STOPPED! ***"
        
        return message, early_stop
    
    def save_checkpoint(self, val_loss, model):
        torch.save(model.state_dict(), self.file_path)
        torch.save(model.state_dict(), self.latest_file_path)
        self.val_loss_min = val_loss

In [5]:
class ClassificationTrainer:
    def __init__(
    self, project_name, model, optimizer, train_data_loader, validation_data_loader, transforms,
    run_time_str, wandb, device, checkpoint_file_path
    ):
        self.project_name = project_name
        self.model = model
        self.optimizer = optimizer
        self.train_data_loader = train_data_loader
        self.validation_data_loader = validation_data_loader
        self.transforms = transforms
        self.run_time_str = run_time_str
        self.wandb = wandb
        self.device = device
        self.checkpoint_file_path = checkpoint_file_path
        
        self.loss_fn = nn.CrossEntropyLoss()

    def do_train(self):
        self.model.train()
        
        loss_train = 0.0
        num_corrects_train = 0
        num_trained_samples = 0
        num_trains = 0
        
        for train_batch in self.train_data_loader:
            input_train, target_train = train_batch
            input_train = input_train.to(device=self.device)
            target_train = target_train.to(device=self.device)
            
            if self.transforms:
                input_train = self.transforms(input_train)
            
            output_train = self.model(input_train)
            
            loss = self.loss_fn(output_train, target_train)
            loss_train += loss.item()
            
            predicted_train = torch.argmax(output_train, dim=1)
            num_corrects_train += torch.sum(torch.eq(predicted_train, target_train)).item()
            
            num_trained_samples += len(input_train)
            num_trains += 1
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        
        train_loss = loss_train / num_trains
        train_accuracy = 100.0 * num_corrects_train / num_trained_samples
        
        return train_loss, train_accuracy

    def do_validation(self):
        self.model.eval()
        
        loss_validation = 0.0
        num_corrects_validation = 0
        num_validated_samples = 0
        num_validations = 0
        
        with torch.no_grad():
            for validation_batch in self.validation_data_loader:
                input_validation, target_validation = validation_batch
                input_validation = input_validation.to(device=self.device)
                target_validation = target_validation.to(device=self.device)
                
                if self.transforms:
                    input_validation = self.transforms(input_validation)
                
                output_validation = self.model(input_validation)
                loss_validation += self.loss_fn(output_validation, target_validation).item()
                
                predicted_validation = torch.argmax(output_validation, dim=1)
                num_corrects_validation += torch.sum(torch.eq(predicted_validation, target_validation)).item()
                
                num_validated_samples += len(input_validation)
                num_validations += 1
        
        validation_loss = loss_validation / num_validations
        validation_accuracy = 100.0 * num_corrects_validation / num_validated_samples
        
        return validation_loss, validation_accuracy

    def train_loop(self):
        early_stopping = EarlyStopping(
            patience=self.wandb.config.early_stop_patience,
            project_name=self.project_name,
            checkpoint_file_path=self.checkpoint_file_path,
            run_time_str=self.run_time_str
        )
        n_epochs = self.wandb.config.epochs
        training_start_time = datetime.now()

        for epoch in range(1, n_epochs + 1):
            train_loss, train_accuracy = self.do_train()

            if epoch == 1 or epoch % self.wandb.config.validation_intervals == 0:
                validation_loss, validation_accuracy = self.do_validation()
    
                elapsed_time = datetime.now() - training_start_time
                epoch_per_second = 0 if elapsed_time.seconds == 0 else epoch / elapsed_time.seconds
    
                message, early_stop = early_stopping.check_and_save(validation_loss, self.model)
    
                print(
                    f"[Epoch {epoch:>3}] "
                    f"T_loss: {train_loss:6.4f}, "
                    f"T_accuracy: {train_accuracy:6.4f} | "
                    f"V_loss: {validation_loss:6.4f}, "
                    f"V_accuracy: {validation_accuracy:6.4f} | "
                    f"{message} | "
                    f"T_time: {strfdelta(elapsed_time, '%H:%M:%S')}, "
                    f"T_speed: {epoch_per_second:4.3f}"
                )
    
                self.wandb.log({
                  "Epoch": epoch,
                  "Training loss": train_loss,
                  "Training accuracy (%)": train_accuracy,
                  "Validation loss": validation_loss,
                  "Validation accuracy (%)": validation_accuracy,
                  "Training speed (epochs/sec.)": epoch_per_second,
                })
    
                if early_stop:
                  break

        elapsed_time = datetime.now() - training_start_time
        print(f"Final training time: {strfdelta(elapsed_time, '%H:%M:%S')}")